# Towers of Hanoi

This problem is derived from the [introductory manual](http://wp.doc.ic.ac.uk/arusso/wp-content/uploads/sites/47/2015/01/clingo_guide.pdf), but written out so that the code is commented in detail about what each exprssion means. Let's first start with a summary of the problem.

## Summary of Task

In this puzzle, you have three pegs and discs of different sizes. The discs start on one peg, stacked so that no disk is on top of another disk that is smaller than it. Your goal is to move all the discs from the peg on the left to the peg on the right, with the following constraints:

1. You can only move a peg on the top of a stack
2. You can only put a peg on top of another one that is larger.

If you were doing this with actual pegs, you would probably think about a sequence of moves. But with Answer Set Programming, the cool part is that you can define the pieces of the problem (the pegs, disks) along with the rules, define the ideal solution, and let the solver do its magic. We will proceed to take the same solution provided in the pdf, and annotate it extensively for you to quickly see, understand, and run.

## 1. Create Objects

Conceptually, let's talk about the objects that we want to model first as facts or atoms. The first are fairly straight forward - we will model the physical:

1. pegs
2. disks

In [26]:
%%clingo --out-ifs=\\n 0

% Create three pegs, a,b, and c. This is akin to peg(a). peg(b). peg(c).
% Putting them together like this is called pooling. We are lazy so we want
% to define three things in one go!
peg(a;b;c).

% Let's do the same for disks. We will have four, with ids 1,2,3 and 4.
% This is also a shortcut notation called an interval.
disk(1..4).

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1
disk(1)
disk(2)
disk(3)
disk(4)
peg(a)
peg(b)
peg(c)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


When the program runs above, we see our intervals and pools flushed out into the full lits of objects. We now have four disks, named by numbers, and three pegs, a b and c. Great!

## 2. Create States

The next set of atoms we want to create is related to states of the game. How does it start? How should it end? We thus want to define:

1. an init state
2. a "yay we're done!" state

Knowing that we have pegs and disks, it follows that the states should include how we expect the pegs and disks to be. For example, we know that we will start with all four disks on peg a. We know that we must finish with all four disks on beg c. So how about we make a rule for each of init and finish that represents a state, along with some number (the ids) of disks on some peg. That might look like this:

In [27]:
%%clingo --out-ifs=\\n 0

% start the game with disks 1, 2, 3, 4 ("one through four") on peg a
init_on(1..4,a).

% finish the game with those same disks one through four on peg c.
finish_on(1..4,c).

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1
init_on(1,a)
init_on(2,a)
init_on(3,a)
init_on(4,a)
finish_on(1,c)
finish_on(2,c)
finish_on(3,c)
finish_on(4,c)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


We can again see when we run the code that our `init_on` and `finish_on` states are actually a bunch of different facts that must all be true to determine that we've started or finished. Okay, great! Just to show you, let's say I was writing my program and I didn't want any facts to print (because I'm not done yet!). I can add `#show.`.

In [28]:
%%clingo --out-ifs=\\n 0

% start the game with disks 1, 2, 3, 4 ("one through four") on peg a
init_on(1..4,a).

% finish the game with those same disks one through four on peg c.
finish_on(1..4,c).
#show.

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1

SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


The command was previously called `#hide.` but it [doesn't seem to exist anymore](https://sourceforge.net/p/potassco/mailman/message/34222978/). Hiding everything is essentially asking it to show nothing in particular, as we did above. Next, let's define some boundaries.

## 3. Create Boundaries

A boundary is a limit, or in the case of the Towers of Hanoi, the maximum number of moves we want to allow. For a solver, we can use this as a strategy to prevent it from finding a infinitely large number of solutions with infinite moves (yeah, yikes!). So let's say that we limit to just 15 moves. That is actually really easy to state, like this: 

```
% We are only allowed 15 moves.
moves(15).
```

## 4. Define Rules

Now it get's fun! Let's pretend that you are staring at three pegs, and the four disks are on the left one. How do you know if you can move one disk to another peg? As a reminder of the problem:

1. You can only move a peg on the top of a stack
2. You can only put a peg on top of another one that is larger.

This means that we need a few rules. We need to define what constitutes a disk being blocked, meaning that we cannot reach it. We also need to define what a valid move looks like, namely moving a disk on the top of a stack to another peg.

### Generate Moves

Let's take a mindset that we are going to generate a whole ton of valid moves, and just have them available for other rules to act on. This is probably one of the harder lines to write, so let's walk through it. First, here is the whole thing:

```
1 { move(D,P,T) : disk(D), peg(P) } 1 :- moves(M), T = 1..M.
```
Let's first notice that the "if" that we recognize (the `:-` is right before `moves(M)`, meaning that the head of the rule is `1 { move(D,P,T) : disk(D), peg(P) } 1`. Whenever you see numbers surrounding some statement in curly brackets, this is a cardinality constaint. Whatever we are stating in those brackets, there can only be one. If you defined other numbers, it would increase the allowed range.  Now let's look at the set of statements inside the curly brackets, namely `move(D,P,T) : disk(D), peg(P)`. This is a set that is expanded using the predicates behind the colons. I was reading it as:

```
% Move D, P, and T, given that D is a disk, and P is a peg.
move(D,P,T) : disk(D), peg(P).
```

Okay now let's add the rule body, everything after the `:-`.

```
% Move D, P, and T, given that D is a disk, and P is a peg IF we have M moves, and 
% some timepoint (T) is between 1 and M.
1 { move(D,P,T) : disk(D), peg(P) } 1 :- moves(M), T = 1..M.
```

If you had to write this, you'd probably start with the first statement in the head `move(D, P, T)` and then realize that you need to ensure that D is a disk and P is a peg (the second part) and a move is only one of these statements (add the curly brackets and cardinality of 1) and then you would think through the IF. "Move disk D, peg P, for some timepoint T IF we have 15 moves, AND (the command) the timepoint can go from 1 through M.
You might not have included the variable T when you first wrote it, but you'd eventually realize you need to scope one move to one timepoint. If you consider that we will are trying to figure out values for pegs and disks, this rule is actually setting the timepoint, T, to be each number between 1 and M and then solving for the peg and disk.

But we can't just give that to the program, because we haven't provided any constraints. This rule would basically let us move any disk to any peg at any time. We haven't told it how to move, or what it means to be blocked. If we try running it, it will run forever (at least as far as I can tell). I interrupted the kernel process below because it seemed to run forever:

In [ ]:
%%clingo --out-ifs=\\n 0

% Create three pegs, (a, b and c) and four disks
peg(a;b;c).
disk(1..4).

% A maximum of 15 moves is allowed
moves(15).

1 { move(D,P,T) : disk(D), peg(P) } 1 :- moves(M), T = 1..M.

## Define Valid Moves

Conceptually, we now have a bunch of options for moves, `move(D,P,T)` to say we are moving a disk relative to a peg for a timepoint, T. The next part is tricky, because we have to define what it means to be at a timepoint. Let's call this `at_timepoint(D, P, T)`. First we might say that, "We are at timepoint zero (the beginning of the game) IF we are in state init on":

```
% We are at timepoint 0 IF we are in state init on.
% If you recall from above, init_on also takes a disk and peg.
at_timepoint(D, P, 0) :- init_on(D,P).
```
And we are at a general timepoint for any given value of T if we move there:

```
% moving a disk changes its location
at_timepoint(D, P, T) :- move(D, P, T).
```

The next rule you'll try to write ensures that a timepoint, T+1, comes after a timepoint T. It also ensures that we haven't reached the maximum number of allowed moves (since we start counting at 0, we go up to T but not equal to it). You might first try this:

```
% We can be at the next timepoint, T+1, if we're at timepont T, and we haven't taken a move for this 
% timepoint yet, and we're not at the maximum number of moves.
at_timepoint(D,P,T+1) :- at_timepoint(D,P,T), not move(D,P,T+1), not moves(T).
```

But there's a problem, and that's the rule `move(D, P, T+1)`. You see, we don't actually care about what peg was relevant for that move. We only care if a disk was moved at that timepoint, to any peg. To fix this rule, we need to write another one that elimiates the variable "D", because we don't care about the disk. This says that "We move a disk at a timepoint given any move of a disk D, at timepoint T for ANY peg (why we replace the variable P with an underscore, `_`.

```
% We move any disk at a timepoint IF we've moved a disk at a time point for any peg, we don't care -> (_)
move(D,T)   :- move(D,_,T).
```

Now we can write this rule more correctly:

```
% We can be at the next timepoint, T+1, if we're at timepont T, and we haven't taken a move for this 
% timepoint yet, and we're not at the maximum number of moves.
at_timepoint(D,P,T+1) :- at_timepoint(D,P,T), not move(D,T+1), not moves(T).
```

## Define Blocked

We're close! So far we've defined what it means to move any disk to any peg at a timepoint, and we also now have temporal constraints that T+1 must come after T. What is missing? The idea of a blocked disk! Right now, there is nothing stopping you from grabbing a disk that is second on a peg. For these next statements, it's also important to remember that disks are numbered based on size, with smaller numbers indicating smaller disks.

```
% A smaller disk (D-1) for some peg P, and the next timepoint, T+1, is blocked IF:
% the current timepoint has a larger disk (D) on the same peg, and we haven't
% finished making all of our moves.

% In simpler terms, the disk below a position is blocked
blocked(D-1,P,T+1) :- at_timepoint(D,P,T), disk(D), not moves(T).
```

Basically, you can't move a disk if there is a larger one on top of it from one timepoint to the next. But what about the same timepoint? Similarity, a smaller disk is blocked for some timepoint T if it's blocked. This sounds weird to read, but we are basically using the same rule to not be rudundant, but changing the value of T.

```
% The disk below a blocked position is also blocked
blocked(D-1,P,T)   :- blocked(D,P,T), disk(D).
```

Both the rules together say that a smaller disk is blocked for the current or next timepoint if our current timepoint has a larger disk on the same peg. I'm not sure if `at_timepoint` is the best descriptor, I think in the first examples they had the rule written as `on(D,P,T)` which more directly says "We are ON this current move."

## Integrity Constraints

Rules that start with `:-` are integiry constraints. They fail whenever all the literals (in the body) are true. This means that these statements cannot be true.

```
% A blocked disk (smaller in size on the same peg) cannot be moved
:- move(D,P,T), blocked(D-1,P,T).

% A disk can only be placed on top of a bigger disk. 
:- move(D,T), at_timepoint(D,P,T-1), blocked(D,P,T).

% We finish when we reach moves M, not at that timepoint (because we start countint at 0)
:- finish_on(D,P), not at_timepoint(D,P,M), moves(M).

% Re-iteration of the first rule to speed up the problem
:- not 1 { at_timepoint(D,P,T) : peg(P) } 1, disk(D), moves(M), T = 1..M.
```

# All Together Now!

Let's put the pieces together and run the full program. We will add a show statement at the end to show moves.

In [ ]:
%%clingo --out-ifs=\\n 0

% Create three pegs, (a, b and c) and four disks
peg(a;b;c).
disk(1..4).

% start the game with disks 1, 2, 3, 4 ("one through four") on peg a
init_on(1..4,a).

% finish the game with those same disks one through four on peg c.
finish_on(1..4,c).


% A maximum of 15 moves is allowed
moves(15).

1 { move(D,P,T) : disk(D), peg(P) } 1 :- moves(M), T = 1..M.

% We are at timepoint 0 IF we are in state init on.
% If you recall from above, init_on also takes a disk and peg.
at_timepoint(D, P, 0) :- init_on(D,P).

% moving a disk changes its location
at_timepoint(D, P, T) :- move(D, P, T).

move(D,T)   :- move(D,_,T).

% We can be at the next timepoint, T+1, if we're at timepont T, and we haven't taken a move for this 
% timepoint yet, and we're not at the maximum number of moves.
at_timepoint(D,P,T+1) :- at_timepoint(D,P,T), not move(D,T+1), not moves(T).

% In simpler terms, the disk below a position is blocked
blocked(D-1,P,T+1) :- at_timepoint(D,P,T), disk(D), not moves(T).

% The disk below a blocked position is also blocked
blocked(D-1,P,T)   :- blocked(D,P,T), disk(D).



% A blocked disk (smaller in size on the same peg) cannot be moved
:- move(D,P,T), blocked(D-1,P,T).

% A disk can only be placed on top of a bigger disk. 
:- move(D,T), at_timepoint(D,P,T-1), blocked(D,P,T).

% We finish when we reach moves M, not at that timepoint (because we start countint at 0)
:- finish_on(D,P), not at_timepoint(D,P,M), moves(M).

% Re-iteration of the first rule to speed up the problem
:- not 1 { at_timepoint(D,P,T) : peg(P) } 1, disk(D), moves(M), T = 1..M.

#show move/3.

And there you have it! We've come up with a solution for moving disks. Could you think of a cool way to visualize this output? If you'd like to improve this tutorial, please [visit the repository](https://github.com/vsoch/clingo-lessons) and open an issue or a pull request. I wrote it very late so who knows where my brain is. :)